# AtomMan Unit Conversion

The atomman package handles unit conversion by building off of the [numericalunits](https://pypi.python.org/pypi/numericalunits) package.  In this manner, units are not 'controlled' but converted at the beginning and end of simulations.  This minimizes overhead and allows for simple code. numericalunits works by defining all units relative to five independent base units (meters, kilograms, seconds, coulombs, kelvins).  In this manner all units for dimensionally correct values will cancel out.  More information can be found in the numericalunits documentation.

The underlying code can be found in unitconvert.py.

__Library imports__

In [1]:
#External library imports
import numpy as np

#atomman imports
import atomman.unitconvert as uc

## 1. Value assignment and retrieval

The units module of atomman contains the functions set_in_units() and get_in_units() that handles unit conversions when data is assigned and retrieved, respectively.  The function arguments are value and unit, where unit can be specified as a string, numerical value, or None.

In [2]:
#Using set_in_units and get_in_units with string unit names
print "Units can be specified as strings:"
print "five_feet = uc.set_in_units(5, 'foot')"
five_feet = uc.set_in_units(5, 'foot')
print "uc.get_in_units(five_feet, 'inch') ->", uc.get_in_units(five_feet, 'inch') 
print 

#If one value is relative to another, the already assigned value can be used
print 'Alternatively, values can be scaled:'
relative_position = np.array([0.5, 0.5, 0.5])
print "relative_position ->", relative_position

a_lattice = uc.set_in_units(3.2, 'angstrom')
print "uc.get_in_units(a_lattice, 'angstrom') ->", uc.get_in_units(a_lattice, 'angstrom')

print "position = uc.set_in_units(relative_position, a_lattice)"
position = uc.set_in_units(relative_position, a_lattice)
print "uc.get_in_units(position, 'angstrom') ->", uc.get_in_units(position, 'angstrom')
print 

#If the unit is None, value is not scaled
print "Units of None does no conversion:"
print "uc.get_in_units(42, None) ->", uc.get_in_units(42, None)
print "uc.set_in_units(42, None) ->", uc.set_in_units(42, None)

Units can be specified as strings:
five_feet = uc.set_in_units(5, 'foot')
uc.get_in_units(five_feet, 'inch') -> 60.0

Alternatively, values can be scaled:
relative_position -> [ 0.5  0.5  0.5]
uc.get_in_units(a_lattice, 'angstrom') -> 3.2
position = uc.set_in_units(relative_position, a_lattice)
uc.get_in_units(position, 'angstrom') -> [ 1.6  1.6  1.6]

Units of None does no conversion:
uc.get_in_units(42, None) -> 42
uc.set_in_units(42, None) -> 42


## 2.  Dictionary of units

Access of the units by string is accomplished by creating a dictionary "unit" containing all of numericalunits defined units.  Note that the default setting for atomman is that all values are converted to working units based on the following dimension settings:

- length in angstroms

- mass in amu

- energy in eV

- charge in e (proton charge)

- temperature in K

In [3]:
print "uc.unit['angstrom'] ->", uc.unit['angstrom']
print "uc.unit['amu'] ->", uc.unit['amu']
print "uc.unit['eV'] ->", uc.unit['eV']
print "uc.unit['e'] ->", uc.unit['e']
print "uc.unit['K'] ->", uc.unit['K']
print
print "uc.unit['m'] ->", uc.unit['m']

uc.unit['angstrom'] -> 1.0
uc.unit['amu'] -> 1.0
uc.unit['eV'] -> 1.0
uc.unit['e'] -> 1.0
uc.unit['K'] -> 1.0

uc.unit['m'] -> 10000000000.0


## 3. Unit string parsing

More complex or custom unit specifications can also be converted from strings using the parse() method.  The parse method works according to the following rules:

1. The argument is broken apart into terms and operators.

2. Allowable operators are: * for multiplication, / for division, ^ for powers, and ().  Normal order of operations is used.  Multiplication must be explicitly specified. 

3. Terms can be numbers (starting with digit, minus sign or decimal point) or strings (starting with letter).  Each term ends when either whitespace or a operator is found. 

4. Number terms are converted into floats

5. String terms are converted using the unit dictionary.

In [4]:
#Testing parse()
print "uc.parse('nm*eV') ->", uc.parse('nm*eV')
print "uc.parse('m^3') ->", uc.parse('m^3')
print "uc.parse('eV/J') ->", uc.parse('eV/J')
print "uc.parse(None) ->", uc.parse(None)
print "uc.parse(1492) ->", uc.parse(1492)
print "uc.parse('1492') ->", uc.parse('1492')

uc.parse('nm*eV') -> 10.0
uc.parse('m^3') -> 1e+30
uc.parse('eV/J') -> 1.602176565e-19
uc.parse(None) -> 1
uc.parse(1492) -> 1492
uc.parse('1492') -> 1492.0


The get_in_units() and set_in_units() call parse() as well.

In [5]:
#Testing get_in_units() and set_in_units() with complex strings
print '1e5 dyn = N = kg*m/s^2:'

newton = uc.set_in_units(1e5, 'dyn')
print "  newton = uc.set_in_units(1e5, 'dyn')"
print "  uc.get_in_units(newton, 'kg*m/s^2') ->", uc.get_in_units(newton, 'kg*m/s^2')
print

print 'pg/(um*us^2) = 1000 Pa = kPa:'

pressure = uc.set_in_units(5, 'pg / (um * us^2)')
print "  pressure = uc.set_in_units(5, 'pg / (um * us^2)')"
print "  uc.get_in_units(pressure, 'kPa') ->", uc.get_in_units(pressure, 'kPa')

1e5 dyn = N = kg*m/s^2:
  newton = uc.set_in_units(1e5, 'dyn')
  uc.get_in_units(newton, 'kg*m/s^2') -> 1.0

pg/(um*us^2) = 1000 Pa = kPa:
  pressure = uc.set_in_units(5, 'pg / (um * us^2)')
  uc.get_in_units(pressure, 'kPa') -> 5.0


## 4. Changing the working units

The units that atomman works in can be changed using the reset_units() function.  Function arguments allow for the specification of units for:

- length 

- mass 

- time

- energy

- charge

__Note #1__: length, mass, time and energy cannot all be specified as they are not independent.

__Note #2__: Temperature is left in K as temperature conversions require both scaling and shifting.

__Note #3__: If less than four dimension terms are specified in reset(), the non-dependent terms will be scaled to the SI standards (length = 'm', mass='kg', time='s', charge='C', temperature='K').

In [6]:
uc.reset_units(length='nm', energy='eV', time='ps', charge='e')
print "uc.unit['nm'] ->", uc.unit['nm']
print "uc.unit['eV'] ->", uc.unit['eV']
print "uc.unit['ps'] ->", uc.unit['ps']
print "uc.unit['e'] ->", uc.unit['e']
print "uc.unit['K'] ->", uc.unit['K']
print
print "uc.unit['angstrom'] ->", uc.unit['angstrom']

uc.unit['nm'] -> 1.0
uc.unit['eV'] -> 1.0
uc.unit['ps'] -> 1.0
uc.unit['e'] -> 1.0
uc.unit['K'] -> 1.0

uc.unit['angstrom'] -> 0.1


The reset_units() function also allows for the numericalunits' standard random working units to be applied simply by calling the function with no arguments, or just a random number seed.  This is useful during development as it offers a check into proper dimension handling. 

From [numericalunits' documentation](https://pypi.python.org/pypi/numericalunits):

_"In a dimensionally-correct calculation, the units all cancel out, so the final answer is deterministic, not random. In a dimensionally-incorrect calculations, there will be random factors causing a randomly-varying final answer."_

In [7]:
uc.reset_units()
print "uc.unit['m'] ->", uc.unit['nm']
print "uc.unit['kg'] ->", uc.unit['eV']
print "uc.unit['s'] ->", uc.unit['ps']
print "uc.unit['C'] ->", uc.unit['e']
print "uc.unit['K'] ->", uc.unit['K']
print
print "uc.unit['angstrom'] / uc.unit['angstrom'] ->", uc.unit['angstrom'] / uc.unit['angstrom']

uc.unit['m'] -> 1.02363081577e-10
uc.unit['kg'] -> 2.73898580607e-20
uc.unit['s'] -> 4.01840950663e-13
uc.unit['C'] -> 2.58565504442e-20
uc.unit['K'] -> 9.87420596949

uc.unit['angstrom'] / uc.unit['angstrom'] -> 1.0
